In [1]:
# imports
from timezonefinder import TimezoneFinder
import math
from datetime import datetime
from pytz import timezone, utc
import xarray as xr
import numpy as np

In [2]:
def sunpos(when, location, refraction = True):
    '''
    sunpos: returns the elevation and azimuth at a given time and location - refraction is optional, True if no value is given
    '''
    def into_range(x, range_min, range_max):
        '''
        used to turn azimuth and elevation from radians to degrees
        '''
        shiftedx = x - range_min
        delta = range_max - range_min
        return (((shiftedx % delta) + delta) % delta) + range_min
    
    # Extract the passed data
    year, month, day, hour, minute, second, timezone = when
    latitude, longitude = location
    # Math typing shortcuts
    rad, deg = math.radians, math.degrees
    sin, cos, tan = math.sin, math.cos, math.tan
    asin, atan2 = math.asin, math.atan2
    # Convert latitude and longitude to radians
    rlat = rad(latitude)
    rlon = rad(longitude)
    # Decimal hour of the day at Greenwich
    greenwichtime = hour - timezone + minute / 60 + second / 3600
    # Days from J2000, accurate from 1901 to 2099
    daynum = (
        367 * year
        - 7 * (year + (month + 9) // 12) // 4
        + 275 * month // 9
        + day
        - 730531.5
        + greenwichtime / 24
    )
    # Mean longitude of the sun
    mean_long = daynum * 0.01720279239 + 4.894967873
    # Mean anomaly of the Sun
    mean_anom = daynum * 0.01720197034 + 6.240040768
    # Ecliptic longitude of the sun
    eclip_long = (
        mean_long
        + 0.03342305518 * sin(mean_anom)
        + 0.0003490658504 * sin(2 * mean_anom)
    )
    # Obliquity of the ecliptic
    obliquity = 0.4090877234 - 0.000000006981317008 * daynum
    # Right ascension of the sun
    rasc = atan2(cos(obliquity) * sin(eclip_long), cos(eclip_long))
    # Declination of the sun
    decl = asin(sin(obliquity) * sin(eclip_long))
    # Local sidereal time
    sidereal = 4.894961213 + 6.300388099 * daynum + rlon
    # Hour angle of the sun
    hour_ang = sidereal - rasc
    # Local elevation of the sun
    elevation = asin(sin(decl) * sin(rlat) + cos(decl) * cos(rlat) * cos(hour_ang))
    # Local azimuth of the sun
    azimuth = atan2(
        -cos(decl) * cos(rlat) * sin(hour_ang),
        sin(decl) - sin(rlat) * sin(elevation),
    )
    # Convert azimuth and elevation to degrees
    azimuth = into_range(deg(azimuth), 0, 360)
    elevation = into_range(deg(elevation), -180, 180)
    # Refraction correction (optional)
    if refraction:
        targ = rad((elevation + (10.3 / (elevation + 5.11))))
        elevation += (1.02 / tan(targ)) / 60
    # Return azimuth and elevation in degrees
    return (round(azimuth, 2), round(elevation, 2))

def get_offset(*, lat, lng):
    """
    returns a location's time zone offset from UTC in minutes.
    """
    tf= TimezoneFinder()
    today = datetime.now()
    tz_target = timezone(tf.certain_timezone_at(lng=lng, lat=lat))
    # ATTENTION: tz_target could be None! handle error case
    today_target = tz_target.localize(today)
    today_utc = utc.localize(today)
    return (today_utc - today_target).total_seconds() / 60

def loc_to_day(lat, lng, time):
    '''
    loc_to_day: takes one latitude, longitude, and time and determines if it is daytime (sunny) or not - 
                the change from day to night (i.e. True to False) shows sunrise and sunset for visa versa
    '''
    location = (lat, lng)
    tz_diff = get_offset(**{"lat": location[0], "lng": location[1]})
    date_str, time_str = np.datetime_as_string(time).split("T")
    year, month, day = [int(x) for x in date_str.split("-")]
    hour, minute, second = [int(float(x)) for x in time_str.split(":")]
    when = (year, month, day, hour, minute, second, tz_diff)
    _, elevation = sunpos(when, location)
    return elevation > 0
    

In [3]:
filename = "IMOS_SOOP-BA_AE_20160412T031347Z_VLMJ_FV02_Investigator-EK60-18_END-20160414T043251Z_C-20210622T061448Z.nc"
ds = xr.open_dataset(filename)
lat = ds.LATITUDE
lng = ds.LONGITUDE
time = ds.TIME

In [7]:
sun = [] # True/False array for day/night for each ping
for i in range(len(time)):
    sun.append(loc_to_day(lat[i], lng[i], time[i]))

In [8]:
ds.assign(SUN = np.array(sun)) # assignes day/night as x-arraay coordinate

<xarray.Dataset>
Dimensions:                                                (DEPTH: 240, EV_FILENAME: 1, SUN: 6569, TIME: 6569)
Coordinates:
  * TIME                                                   (TIME) datetime64[ns] ...
  * DEPTH                                                  (DEPTH) float64 2....
  * EV_FILENAME                                            (EV_FILENAME) |S256 ...
  * SUN                                                    (SUN) bool True .....
Data variables: (12/58)
    LATITUDE                                               (TIME) float64 -50...
    LATITUDE_quality_control                               (TIME) float32 1.0...
    LONGITUDE                                              (TIME) float64 150...
    LONGITUDE_quality_control                              (TIME) float32 1.0...
    mean_height                                            (TIME, DEPTH) float64 ...
    mean_height_quality_control                            (TIME, DEPTH) float32 ...
    ...                                                     ...
    upper_mesopelagic                                      (TIME) float64 -71...
    upper_mesopelagic_quality_control                      (TIME) float32 2.0...
    lower_mesopelagic                                      (TIME) float64 -74...
    lower_mesopelagic_quality_control                      (TIME) float32 2.0...
    day                                                    (TIME) float64 1.0...
    day_quality_control                                    (TIME) float32 0.0...
Attributes: (12/156)
    abstract:                                                    IMOS Bioacou...
    acknowledgement:                                             Any users of...
    author:                                                      Amy Nau
    author_email:                                                Amy.Nau@csir...
    calibration_accuracy_estimate:                               0.5
    calibration_acquisition_method:                              Standard sph...
    ...                                                          ...
    transit_start_locality:                                      50.9S 150.7E
    units:                                                       signed decim...
    uplimit:                                                     2.5
    westlimit:                                                   142.025818169
    zunits:                                                      m
    SUN:                                                         [ True  True...